In [1]:
%%bash
which python

/home/adm2/anaconda2/bin/python


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
#import urllib2
import csv
import sys, os
import consts

Functions taken from `xml2tree.py` script

In [3]:
def parse_ins(x):
    n_ins = 0
    d = x
    pos, ins = d.split('.')
    # problem about 'XC' to be solved, so far
    # those events are not considered
    try:
        n_ins = int(ins[0])
        nat_ins = ins[1:]
        return '.'.join([pos, nat_ins*n_ins])
    except ValueError:
        pass

def retro_handle(i):
    # this function should be called before checking if the mutation is either a transition or a transversion.
    u = mut_handle(i.replace('!', ''))
    # an even number of "!" means that the change is actually occurring
    if i.count('!')%2 == 0:
        n = u
    # an odd number of "!" means that there is actually a retromutation
    else:
        n = u+'!'
    # check if the mutation is a transition or a transversion
    return n
 
def mut_handle(i):
    #print i
    # preliminar check of deletion
    if "." in i:
        n = parse_ins(i)
        if n is None:
            pass
    elif "d" in i:
        # in new phylotree versions deletions of single nts
        # are preceded by nt but have to be removed
        if i[0].isdigit() == False:
            n = i[1:]
        else:
            n = i
    else:
        x, y = [i[0].upper(), i[-1].upper()]
        if (x in consts.PUR and y in consts.PUR) or (x in consts.PYR and y in consts.PYR):
            # transition, from "G100A" get "100"
            n = i[1:-1]
        elif (x in consts.PUR and y in consts.PYR) or (x in consts.PYR and y in consts.PUR):
            # transversion, from "C100A" get "100A"
            n = i[1:].upper()
    return n
 

def events_parsing_string(string):
    # cfr events_parsing
    n_events = []
    events = string.split(',')
    # events between brackets are discarded
    events = [i for i in events if i[0] != '(']
    for e in events:
        # note: the function "retro_handle" handles all mutation types
        if "X" in e:
            print "%s, mutation ignored." % e
            continue
        e = retro_handle(e)
        if e is None:
            continue
        n_events.append(e)
    return n_events

def event_parsing(event):
    e = event
    # cfr events_parsing
    #n_events = []
    #events = string.split(',')
    # events between brackets are discarded
    #events = [i for i in events if i[0] != '(']
    #for e in events:
    # note: the function "retro_handle" handles all mutation types
    if "X" in e or "(" in e:
        print "%s, mutation ignored." % e
        E = ''
    else:
        E = retro_handle(e)
    if E is None:
        E = ''
    #    n_events.append(e)
    return E

In [18]:
def assignNoLabelStatus(s):
    status = "Label"
    s = s.strip(',').split(',')
    t = s[0]
    if len(t) > 1:
        # case to discard, eg C123G
        if t[0] in list('ACGT') and t[-1] in list('ACGT'):
            status = "NoLabel"
        # case to discard, eg C123G!
        elif t[0] in list('ACGT') and t[-1] == "!":
            status = "NoLabel"
        elif t[0].isalpha() == False:
            status = "NoLabel"
        # This last case replaces the next ones
        # case to discard, eg deletion 8123-8234d
        #elif t[0].isdigit() and t[-1] == "d":
        #    status = "NoLabel"
        # eg (C64T)
        #elif t[0] == "(":
        #    status = "NoLabel"
    return status

def checkIfAC(s):
    if len(s) > 0:
        if s[0].isalpha() and s[-1].isdigit(): # field is an AC
            return True
        elif s == "reserved":
            return True
        else:
            return False
    else:
        return False

In [19]:
#url = 'http://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47401&view=1'
#html = urllib2.urlopen(url).read()
html = open('mtDNA_tree_Build_17.onlytable.htm', 'r').read()
#print(repr(html))

In [20]:
soup = BeautifulSoup(html, 'lxml')

In [21]:
table = soup.find_all("table")[0]

In [22]:
rows = table.find_all("tr")

In [23]:
# col_normal = rows[6]

In [24]:
csv_list = []
o = open('phylotree_17_190121.csv', 'w')
for row in rows:
    try:
        #print "A row starts here"
        s = ""
        columns = row.find_all('td')
        #print columns
        #for column in columns:
        for x, column in enumerate(columns):
            #column_text = [event_parsing(f.replace("\A0", "")) for f in column.get_text().split()]
            column_text = [f.replace("\A0", "") for f in column.get_text().split()]
            column_text = [f for f in column_text if f != ""]
            #print column_text
            #if len(column_text) != 0:
            #    filled_field = True
            #print "{} field: {}, status: {}".format(x, column_text, filled_field)
            column_text = ','.join(column_text)
            #print column.get_text().split()
            #column_text = ','.join(column.get_text().split())
            s += "{},".format(column_text)
            #s = s.replace("\A0", "")
            #print s
        filled_field = False
        if s.strip(',') != "":
            #print(s)
            #print(s.rstrip(','))
            s_list = s.rstrip(',').split(',')#[:-1]
            # but first:
            # Delete ACs at the end
            while checkIfAC(s_list[-1]) == True:
                #print("Gonna reduce this: {}".format(s_list))
                s_list = s_list[:-1]
            #print(s_list)
            # Check if it's a NoLabel
            filled_field_occurrences = 0
            new_s_list = []
            print "OLD S: {}".format(','.join(s_list))
            for x,i in enumerate(s_list):
                if i != "": # leading fields
                    #print "Non empty field: {}".format(i)
                    # recognize haplogroup
                    filled_field_occurrences += 1
                    if filled_field_occurrences == 1:
                        if i == 'L0a1a' or i == "L0":
                            pass
                            #print s
                        status = assignNoLabelStatus(s)
                        if status == "NoLabel":
                            #print("Found NoLabel at {}".format(','.join(s_list)))
                            #s_list[x-1] = status
                            new_s_list = new_s_list[:-1]
                            new_s_list.append(status)
                            i = event_parsing(i)
                            if i != "":
                                new_s_list.append(i)
                            #break
                        else:
                            new_s_list.append(i)
                    else:
                        i = event_parsing(i)
                        if i == "C5899.d!":
                            print i
                            i = "5899.Cd!"
                        if i != "":
                            new_s_list.append(i)
                        #s_list[x] = i
                        #print s_list
                        # This are haplogroups, so check event.
                        #print(s_list)
                        #try:
                else:
                    #print "Empty field"
                    new_s_list.append(i)
            #while s_list[-1] == "":
            #    s_list = s_list[:-1]
            while new_s_list[-1] == "":
                new_s_list = new_s_list[:-1]
            #except:
    #            print("Error for:\n{}".format(s_list))
                #print(s_list)
            new_s = ','.join(new_s_list)
            print "NEW S: {}\n".format(new_s)
            if status == "NoLabel":
                pass
                #print new_s
            csv_list.append(new_s)
            o.write(new_s+"\n")
    except Exception, e:
        print e
        print "{}".format(columns)

o.close()

OLD S: L0,G263A,C1048T,C3516a,T5442C,T6185C,C9042T,A9347G,G10589A,G12007A,A12720G
NEW S: L0,263,1048,3516A,5442,6185,9042,9347,10589,12007,12720

OLD S: ,L0a'b'f'g'k,A189G,T4586C,C9818T,T16172C
NEW S: ,L0a'b'f'g'k,189,4586,9818,16172

OLD S: ,,L0a'b'f'g,G73A,G185A,C195T,A263G!,A2245G,C5603T,A11641G,C15136T,G15431A
NEW S: ,,L0a'b'f'g,73,185,195,263!,2245,5603,11641,15136,15431

OLD S: ,,,L0a'b'g,A93G,(A95c),T236C,C8428T,A8566G,G9755A,C16148T
(A95c), mutation ignored.
NEW S: ,,,L0a'b'g,93,236,8428,8566,9755,16148

OLD S: ,,,,L0a'g,G11176A,C16188g
NEW S: ,,,,L0a'g,11176,16188G

OLD S: ,,,,,L0a,C146T,G5231A,G5460A,T14308C,T16278C,C16320T
NEW S: ,,,,,L0a,146,5231,5460,14308,16278,16320

OLD S: ,,,,,,L0a1'4,C16168T
NEW S: ,,,,,,L0a1'4,16168

OLD S: ,,,,,,,L0a1,T5096C
NEW S: ,,,,,,,L0a1,5096

OLD S: ,,,,,,,,L0a1a,(C64T),T3866C,,,,,,,,,,,,,,
(C64T), mutation ignored.
NEW S: ,,,,,,,,L0a1a,3866

OLD S: ,,,,,,,,,,A200G,,,,,,,,,,,,,,
NEW S: ,,,,,,,,,NoLabel,200

OLD S: ,,,,,,,,,,L0a1a1,T2759C,,,,,

NEW S: ,,,,,,,,,L2b1a,146,16355

OLD S: ,,,,,,,,,,L2b1a2,A7569G,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,L2b1a2,7569

OLD S: ,,,,,,,,,,L2b1a3,G8856A,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,L2b1a3,8856

OLD S: ,,,,,,,,,,L2b1a4,G12406A,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,L2b1a4,12406

OLD S: ,,,,,,,,,L2b1b,A385G,A6629G,,,,,,,,,,,,,
NEW S: ,,,,,,,,,L2b1b,385,6629

OLD S: ,,,,,,,,L2b2,T6614C,A6806G,T8503C,,,,,,,,,,,,,
NEW S: ,,,,,,,,L2b2,6614,6806,8503

OLD S: ,,,,,,,,,L2b2a,G709A,G8790A,A9350G,A13966G,C14407T,C16354T,,,,,,,,,,
NEW S: ,,,,,,,,,L2b2a,709,8790,9350,13966,14407,16354

OLD S: ,,,,,,,,L2b3,15944.1T
NEW S: ,,,,,,,,L2b3,15944.T

OLD S: ,,,,,,,,,L2b3a,C4185T,G5744A,A8925G,G14544A,A15236G,G15326A,,,,,,,,,,
NEW S: ,,,,,,,,,L2b3a,4185,5744,8925,14544,15236,15326

OLD S: ,,,,,,,,,L2b3b,T2626C,,,,,,,,,,,,,,
NEW S: ,,,,,,,,,L2b3b,2626

OLD S: ,,,,,,,,,L2b3c,T12011C,,,,,,,,,,,,,,
NEW S: ,,,,,,,,,L2b3c,12011

OLD S: ,,,,,,,L2c,A93G,C325T,T680C,G709A,T3200a,G13928c,G13958c,C15849T,,,,,,,,,,,,
NEW S: ,,,,,,,L2c,93,32

OLD S: ,,,,,,,,,,,T199C
NEW S: ,,,,,,,,,,NoLabel,199

OLD S: ,,,,,,,,,,,M38d,T6297C,G7356A,C14944T,T16093C,C16184T,C16260a,C16266T,,,,,,,
NEW S: ,,,,,,,,,,,M38d,6297,7356,14944,16093,16184,16260A,16266

OLD S: ,,,,,,,,,,,M38e,A488G,A2833G,C4654T,T6815C,8289.1CCCCCTCTA,G9438A,A14706G,T16311C!,,,,,,
NEW S: ,,,,,,,,,,,M38e,488,2833,4654,6815,8289.CCCCCTCTA,9438,14706,16311!

OLD S: ,,,,,,,,M30,T195a,G15431A,,,,,,,,,,,,,,
NEW S: ,,,,,,,,M30,195A,15431

OLD S: ,,,,,,,,,M30a,G513A,,,,,,,,,,,,,,
NEW S: ,,,,,,,,,M30a,513

OLD S: ,,,,,,,,,,M30a1,G6366A,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,M30a1,6366

OLD S: ,,,,,,,,,,M30a2,T11935C,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,M30a2,11935

OLD S: ,,,,,,,,,M30b,T152C!,G5147A,C16278T!,T16311C!,,,,,,,,,,,,
NEW S: ,,,,,,,,,M30b,152!,5147,16278!,16311!

OLD S: ,,,,,,,,,M30c,T146C!,A12234G,,,,,,,,,,,,,
NEW S: ,,,,,,,,,M30c,146!,12234

OLD S: ,,,,,,,,,,M30c1,A16166d,,,,,,,,,,,,,
NEW S: ,,,,,,,,,,M30c1,16166d

OLD S: ,,,,,,,,,,,M30c1a,C16069T,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,M30

OLD S: ,,,,,,,,,,,E1a1,T6620C,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,E1a1,6620

OLD S: ,,,,,,,,,,,,E1a1a,T14766C,,,,,,,,,,,
NEW S: ,,,,,,,,,,,,E1a1a,14766

OLD S: ,,,,,,,,,,,,,E1a1a1,C16291T,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,E1a1a1,16291

OLD S: ,,,,,,,,,,,,,,E1a1a1a,T8843C,,,,,,,,,
NEW S: ,,,,,,,,,,,,,,E1a1a1a,8843

OLD S: ,,,,,,,,,,,,,,E1a1a1b,T131C,A8577G,,,,,,,,
NEW S: ,,,,,,,,,,,,,,E1a1a1b,131,8577

OLD S: ,,,,,,,,,,,,,,,E1a1a1b1,C9020T,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,E1a1a1b1,9020

OLD S: ,,,,,,,,,,,,,,,E1a1a1b2,C4102a,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,E1a1a1b2,4102A

OLD S: ,,,,,,,,,,,,,,E1a1a1c,A723t,T9861C,,,,,,,,
NEW S: ,,,,,,,,,,,,,,E1a1a1c,723T,9861

OLD S: ,,,,,,,,,,,,E1a1b,A373G,,,,,,,,,,,
NEW S: ,,,,,,,,,,,,E1a1b,373

OLD S: ,,,,,,,,,,,,,E1a1b1,T6680C,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,E1a1b1,6680

OLD S: ,,,,,,,,,,,,,E1a1b2,T16324C,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,E1a1b2,16324

OLD S: ,,,,,,,,,,,,,E1a1b3,T16223C,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,E1a1b3,16223

OLD S: ,,,,,,,,,,,,,E1a1b4,G16255A,,,,,,,


OLD S: ,,,,,,,M58,A3209G,G5460A,,,,,,,,,,,,,,,
NEW S: ,,,,,,,M58,3209,5460

OLD S: ,,,,,,,M59,A249G,G9380A,T10256C,C11140T,G14040A,T16140C,C16278T!,,,,,,,,,,,,
NEW S: ,,,,,,,M59,249,9380,10256,11140,14040,16140,16278!

OLD S: ,,,,,,,M60,G7912A,C8345T
NEW S: ,,,,,,,M60,7912,8345

OLD S: ,,,,,,,,M60a,A16284G,G16319A
NEW S: ,,,,,,,,M60a,16284,16319

OLD S: ,,,,,,,,,M60a1,C15760T,,,,,,,,,,,,,,
NEW S: ,,,,,,,,,M60a1,15760

OLD S: ,,,,,,,,,M60a2,T2442C,A6728G,C15910T,,,,,,,,,,,,
NEW S: ,,,,,,,,,M60a2,2442,6728,15910

OLD S: ,,,,,,,,M60b,A189G,G3592A,C3894T,T4047C,G5471A,G7853A,G8723A,C16266T,T16357C,,,,,,,,,,
NEW S: ,,,,,,,,M60b,189,3592,3894,4047,5471,7853,8723,16266,16357

OLD S: ,,,,,,,M62'68,C150T,T4561C,G7664A
NEW S: ,,,,,,,M62'68,150,4561,7664

OLD S: ,,,,,,,,M62,G2735A,A3511G,A8149G,G13708A,A15510G,A15520G,T15629C,T15721C,C16295T
NEW S: ,,,,,,,,M62,2735,3511,8149,13708,15510,15520,15629,15721,16295

OLD S: ,,,,,,,,,M62a,T146C!,T310C,C4763T,T9935C,C16147T,,,,,,,,,,,,
NEW S: ,,,,,,,,,M

...

NEW S: ,,,,,,,,,,,,H76a,152!,16093

OLD S: ,,,,,,,,,,,H77,291.1A,G8387A,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H77,291.A,8387

OLD S: ,,,,,,,,,,,H78,C7002T,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H78,7002

OLD S: ,,,,,,,,,,,H79,A12397G,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H79,12397

OLD S: ,,,,,,,,,,,,H79a,G3010A,,,,,,,,,,,
NEW S: ,,,,,,,,,,,,H79a,3010

OLD S: ,,,,,,,,,,,H80,G2361A,T16311C!,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H80,2361,16311!

OLD S: ,,,,,,,,,,,H81,G15172A,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H81,15172

OLD S: ,,,,,,,,,,,,H81a,G2056A,G3421A,G7382A,C9677T,C12063T,,,,,,,,
NEW S: ,,,,,,,,,,,,H81a,2056,3421,7382,9677,12063

OLD S: ,,,,,,,,,,,H82,A16220G,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H82,16220

OLD S: ,,,,,,,,,,,H83,A16212G,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H83,16212

OLD S: ,,,,,,,,,,,H84,G1888A
NEW S: ,,,,,,,,,,,H84,1888

OLD S: ,,,,,,,,,,,H86,T1809C,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H86,1809

OLD S: ,,,,,,,,,,,H87,A8188G,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H87,8188

OLD S: ,,,,,,,,,,,H88,A8596G,,,,,,,,,,,,
NEW S: ,,,,,,,,,,,H

NEW S: ,,,,,,,,,,,,,T2b1,14016

OLD S: ,,,,,,,,,,,,,T2b2,C11242g,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,T2b2,11242G

OLD S: ,,,,,,,,,,,,,,T2b2b,C16304T!,,,,,,,,,
NEW S: ,,,,,,,,,,,,,,T2b2b,16304!

OLD S: ,,,,,,,,,,,,,,,T2b2b1,A12171G,C16192T,,,,,,,
NEW S: ,,,,,,,,,,,,,,,T2b2b1,12171,16192

OLD S: ,,,,,,,,,,,,,T2b3,A10750G,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,T2b3,10750

OLD S: ,,,,,,,,,,,,,,,C151T,,,,,,,,,
NEW S: ,,,,,,,,,,,,,,NoLabel,151

OLD S: ,,,,,,,,,,,,,,,T2b3a,T152C!,A5656G,C16292T,,,,,,,
NEW S: ,,,,,,,,,,,,,,,T2b3a,152!,5656,16292

OLD S: ,,,,,,,,,,,,,,,,T2b3a1,55.1T,T57C,T59C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,T2b3a1,55.T,57,59

OLD S: ,,,,,,,,,,,,,,,T2b3c,T7150C,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,T2b3c,7150

OLD S: ,,,,,,,,,,,,,,,T2b3d,T199C,G14544A,,,,,,,
NEW S: ,,,,,,,,,,,,,,,T2b3d,199,14544

OLD S: ,,,,,,,,,,,,,,,T2b3e,C16187T!,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,T2b3e,16187!

OLD S: ,,,,,,,,,,,,,,T2b3b,A13722G,,,,,,,,,
NEW S: ,,,,,,,,,,,,,,T2b3b,13722

OLD S: ,,,,,,,,,,,,,T2b4,A9254G,,,,,,,,,,
NEW S: ,,,

NEW S: ,,,,,,,,,,,,,,,,B4a1a1ab,9809

OLD S: ,,,,,,,,,,,,,,,,B4a1a1ac,G8790A,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,B4a1a1ac,8790

OLD S: ,,,,,,,,,,,,,,,,B4a1a1ad,G5460A,A15061G,,,,,,
NEW S: ,,,,,,,,,,,,,,,,B4a1a1ad,5460,15061

OLD S: ,,,,,,,,,,,,,,,,B4a1a1ae,T7058C,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,B4a1a1ae,7058

OLD S: ,,,,,,,,,,,,,,,,B4a1a1af,A16212G,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,B4a1a1af,16212

OLD S: ,,,,,,,,,,,,,,,B4a1a2,T4733C,G6366A,T12519C,,,,,,,
NEW S: ,,,,,,,,,,,,,,,B4a1a2,4733,6366,12519

OLD S: ,,,,,,,,,,,,,,,B4a1a3,A14449G,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,B4a1a3,14449

OLD S: ,,,,,,,,,,,,,,,,B4a1a3a,C16223T!,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,B4a1a3a,16223!

OLD S: ,,,,,,,,,,,,,,,,,B4a1a3a1,A8525G,,,,,,
NEW S: ,,,,,,,,,,,,,,,,,B4a1a3a1,8525

OLD S: ,,,,,,,,,,,,,,,,,,B4a1a3a1a,T195C!,T14290C,,,,
NEW S: ,,,,,,,,,,,,,,,,,,B4a1a3a1a,195!,14290

OLD S: ,,,,,,,,,,,,,,,B4a1a4,C4025T,G4769A,,,,,,,
NEW S: ,,,,,,,,,,,,,,,B4a1a4,4025,4769

OLD S: ,,,,,,,,,,,,,,,B4a1a5,G4048A,,,,,,,,
NEW S: ,,,,,,,,,,,,,

OLD S: ,,,,,,,,,,,,,,,,U5a1a1c,A6905G,T13015C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U5a1a1c,6905,13015

OLD S: ,,,,,,,,,,,,,,,,,T16362C,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,NoLabel,16362

OLD S: ,,,,,,,,,,,,,,,,,U5a1a1d,G185A,T204C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,,U5a1a1d,185,204

OLD S: ,,,,,,,,,,,,,,,,,,U5a1a1d1,A189G,,,,,
NEW S: ,,,,,,,,,,,,,,,,,,U5a1a1d1,189

OLD S: ,,,,,,,,,,,,,,,,U5a1a1e,T9682C,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U5a1a1e,9682

OLD S: ,,,,,,,,,,,,,,,,U5a1a1g,C4224T,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U5a1a1g,4224

OLD S: ,,,,,,,,,,,,,,,,U5a1a1i,C3388a,,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U5a1a1i,3388A

OLD S: ,,,,,,,,,,,,,,,U5a1a2,573.XC,C12346T
573.XC, mutation ignored.
NEW S: ,,,,,,,,,,,,,,,U5a1a2,12346

OLD S: ,,,,,,,,,,,,,,,,U5a1a2a,A5319G,A6629G,T6719C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U5a1a2a,5319,6629,6719

OLD S: ,,,,,,,,,,,,,,,,,U5a1a2a1,T6293C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,,U5a1a2a1,6293

OLD S: ,,,,,,,,,,,,,,,,,,U5a1a2a1a,A10972G,,,,,
NEW S: ,,,,,,,,,,,,,,,,,,U5a1a2a1a,10972

OLD S: ,,,,,,,,,,,,,

OLD S: ,,,,,,,,,,,,,,,,,U4b1b1a,C16111T,T16140C,,,,,
NEW S: ,,,,,,,,,,,,,,,,,U4b1b1a,16111,16140

OLD S: ,,,,,,,,,,,,,,,,,U4b1b1b,C146T!!,A8308G,A9389G,A10819G,A15373G,A15758G,,
NEW S: ,,,,,,,,,,,,,,,,,U4b1b1b,146,8308,9389,10819,15373,15758

OLD S: ,,,,,,,,,,,,,,,,,,T16311C!,,,,,,
NEW S: ,,,,,,,,,,,,,,,,,NoLabel,16311!

OLD S: ,,,,,,,,,,,,,,,,,,U4b1b1c,T2483C,,,,,
NEW S: ,,,,,,,,,,,,,,,,,,U4b1b1c,2483

OLD S: ,,,,,,,,,,,,,,,,,U4b1b1d,G5262A,,,,,,
NEW S: ,,,,,,,,,,,,,,,,,U4b1b1d,5262

OLD S: ,,,,,,,,,,,,,,,,U4b1b2,C146T!!,C11788T,G15326A,T15514C,,,,,
NEW S: ,,,,,,,,,,,,,,,,U4b1b2,146,11788,15326,15514

OLD S: ,,,,,,,,,,,,,U4b2,T16136C,,,,,,,,,,
NEW S: ,,,,,,,,,,,,,U4b2,16136

OLD S: ,,,,,,,,,,,,,,U4b2a,G15883A,,,,,,,,,
NEW S: ,,,,,,,,,,,,,,U4b2a,15883

OLD S: ,,,,,,,,,,,,,,,U4b2a1,C16278T!,,,,,,,,
NEW S: ,,,,,,,,,,,,,,,U4b2a1,16278!

OLD S: ,,,,,,,,,,,,,,,,U4b2a1a,T8433C,T14178C,,,,,,
NEW S: ,,,,,,,,,,,,,,,,U4b2a1a,8433,14178

OLD S: ,,,,,,,,,,,,,U4b3,A215G,,,,,,,,,,
NEW S: ,,,,,,,,,,,

In [25]:
it = 'phylotree_17_190121.csv'

In [26]:
ot = 'phylotree_17_190127.csv'

In [27]:
f = open(it,'r')
f = f.readlines()
o = open(ot,'w')

In [28]:
dic_h = {}
c = 0
for branch in f:
	branch = branch.split(',')
	dic_h[c]=''
	new_line = ""
	for x in xrange(len(branch)):
		if branch[x] == '':
			pass
		else:
			if branch[x] != "NoLabel" and branch[x][0].isupper():
				dic_h[c] = (x,branch[x])
				#print dic_h[c],c,branch
				new_line = map(lambda x:str(x),branch)
				new_line = ','.join(new_line).strip()
				#o.write(new_line+'\n')
				break
			elif branch[x] == "NoLabel":
				print ','.join(branch).strip()
				#print 'Here is the problem',c,x
				z = c-1
				while z > 0:
					#print dic_h[z],z,branch[x],x
					if dic_h[z][0] < x:
						distance = str(c-z)
						#print "distance",distance
						new_label = dic_h[z][1]+"_"+str(branch[x+1]).strip()
						dic_h[c] = (x,new_label)
						branch[x] = new_label
						new_line = map(lambda x:str(x),branch)
						new_line = ','.join(new_line).strip()
						break
						#o.write(new_line+'\n')
					else:
						z = z-1
			else:
				pass
	if new_line == "":
		sys.exit("{} yielded an empty new line.".format(branch))
	else:
		if "_" in new_line: print new_line + '\n'
		o.write(new_line+'\n')
	c += 1

,,,,,,,,,NoLabel,200
,,,,,,,,,L0a1a_200,200

,,,,,,,,NoLabel,16293
,,,,,,,,L0a1_16293,16293

,,,,,,NoLabel,152
,,,,,,L0d1b1_152,152

,,,,,NoLabel,189
,,,,,L1b1a_189,189

,,,,,,,,NoLabel,198!
,,,,,,,,L1c1a_198!,198!

,,,,,,,,NoLabel,16189!
,,,,,,,,L2a1_16189!,16189!

,,,,,,,,,,NoLabel,143
,,,,,,,,,,L2a1b_143,143

,,,,,,,,NoLabel,143
,,,,,,,,L2a1_143,143

,,,,,,,,,,NoLabel,16086
,,,,,,,,,,L2a1c_16086,16086

,,,,,,,,,,NoLabel,16129!
,,,,,,,,,,L2a1c_16129!,16129!

,,,,,,,,,NoLabel,16309!
,,,,,,,,,L2a1_143_16309!,16309!

,,,,,,,,,NoLabel,16189!
,,,,,,,,,L2a1_143_16189!,16189!

,,,,,,,,,,NoLabel,16309!
,,,,,,,,,,L2a1_143_16189!_16309!,16309!

,,,,,,,NoLabel,16129!
,,,,,,,L2d_16129!,16129!

,,,,,,,NoLabel,709
,,,,,,,L3a_709,709

,,,,,,,,,,NoLabel,152!
,,,,,,,,,,L3b1a_152!,152!

,,,,,,,,,,NoLabel,16124!
,,,,,,,,,,L3b1a_16124!,16124!

,,,,,,,,,,NoLabel,16292
,,,,,,,,,,L3f1b_16292,16292

,,,,,,,,,,,NoLabel,150
,,,,,,,,,,,L3f1b_16292_150,150

,,,,,,,,,,NoLabel,152!
,,,,,,,,,,L3e2b_152!,152!

,,,,

In [29]:
o.close()

In [30]:
import getopt, sys, re, os, glob, csv
from classifier import tree, NGclassify, consts, datatypes, parse_mhcs
from bioinf.seqs import SeqList
import io_modules.csv
import io_modules.old_table
import io_modules.serialize
import pandas as pd
import os.path
sys.setrecursionlimit(100000)

In [31]:
csvfile = 'phylotree_17_190127.csv'
out_fname = 'haplogroups'

In [32]:
#write pickle file for MToolBox
tree_file = csv.reader(open(csvfile, 'rb'))

pickle_fname = csvfile + '.pickle'
aplo_list = io_modules.csv.parse_csv(tree_file)
htree = tree.HaplogroupTree(aplo_list=aplo_list)
pickle_file = open(pickle_fname, 'wb')
pickle_file.write(htree.serialize())
pickle_file.close()
#write out alleles and haplogroups defined for HmtDB in csv file
pickle_file = pickle_fname
out_file = out_fname + '.csv'
htree = tree.HaplogroupTree(pickle_data=open(pickle_file, 'rb').read())
fh = csv.writer(open(out_file, 'wb'))
for haplo_name in htree:
	io_modules.old_table.write_haplogroup(fh, '', htree[haplo_name])

#write haplogrups.txt tab delimited file for MToolBox
out_file2 = out_fname + '.txt'
hap_file = pd.read_csv(out_file,sep=',',header=None)
subset = hap_file[[0,1,3]]
subset.dropna(inplace=True)
alleles = subset[1].astype(str).str.cat(subset[3])
subset.insert(1,'Allele',alleles)
subset = subset[[0,'Allele']]
subset.columns=['haplogroup_code','POSITIONnucleotidic_change']
subset.to_csv(out_file2,sep='\t',header=True,index=None)


CIAO
M7c1c3(1)
[Transition(769), Transition(15236), Transition(3606), Transition(16129), Transition(4104), Transition(3594), Transition(5442), Transition(11665), Transition(15043), Transition(8468), Transition(16278), Transition(152), Transition(15301), Transition(10400), Transition(10915), Transition(10664), Transition(13105), Transition(16311), Transition(7256), Transversion(825), Transition(10810), Transition(12091), Transition(11914), Transition(14783), Transition(10688), Transition(13506), Transition(195), Transition(2885), Transition(2758), Transition(199), Transition(6455), Transition(8655), Transition(13650), Transition(4312), Transition(13276), Transition(9824), Transition(7521), Transition(16187), Transition(16230), Transition(4071), Transition(489), Transition(16362), Transition(16189), Transition(4850), Transition(247), Transition(1018), Transition(7146)]
CIAO
M7c1c2(2)
[Transition(769), Transition(15236), Transition(3606), Transition(16129), Transition(4104), Transition(35

CIAO
H3_16189!(1)
[Transition(16256), Transition(143), Transition(16400), Transition(2706), Transition(2836), Transition(14766), Transition(6935), Transition(3744), Transition(12705), Transition(808), Transition(9518), Transition(3766), Transition(12346), Transition(11719), Transition(73), Transition(16207), Transition(9809), Transition(466), Transition(16083), Transition(11809), Transition(214), Transition(3421), Transition(16223), Transition(16355), Transition(7269), Transition(6374), Transition(103), Transition(8552), Transition(6632), Transition(7028), Transition(3145), Transition(6776), Transition(6267)]
CIAO
K2(2)
[Transition(8448), Transition(15247), Transition(10771), Transition(12308), Transition(3480), Transition(16153), Transition(9114), Transition(11299), Transition(10550), Transition(13759), Transversion(72), Transition(11467), Transition(14798), Transition(210), Transition(12372), Transition(14167), Transition(5082), Transition(13711), Transition(9055), Transition(16224),

CIAO
L3f2b(7)
[Transition(769), Transition(16129), Transition(11016), Transition(3594), Transition(7256), Transition(146), Transition(8468), Transition(16278), Transition(152), Transition(15514), Transition(15301), Transition(10915), Transition(13105), Transition(10664), Transition(15658), Transition(4104), Transition(12338), Transversion(825), Transition(10810), Transition(16187), Transition(11914), Transition(10688), Transition(13506), Transition(195), Transition(3396), Transition(2885), Transition(2758), Deletion(15944), Transition(8655), Transition(16209), Transition(13650), Transition(4312), Transition(13276), Transition(4218), Transition(5342), Transition(7521), Transition(16230), Transition(15466), Transition(16189), Insertion(374), Transition(247), Transition(1018), Transition(7146), Transition(2442)]
CIAO
U8b1a2a(2)
[Transition(8448), Transition(16259), Transition(15247), Transition(6546), Transition(1811), Transition(12308), Transition(3480), Transition(16153), Transition(911

CIAO
K1b2a2a(1)
[Transition(8448), Transition(15247), Transition(1811), Transition(12308), Transition(16153), Transition(3480), Transition(5913), Transition(9114), Transition(11299), Transition(1189), Transition(10550), Transition(9055), Transition(13759), Transversion(12738), Transversion(72), Transition(11467), Transition(14798), Transition(210), Transition(12372), Transition(14167), Transition(5082), Transition(13711), Transition(14687), Transition(16224), Transition(9698), Transition(12771), Transition(10771), Transition(14071), Transversion(7930)]
CIAO
B4c1c_16311!(1)
[Transition(8448), Transition(15247), Transition(10771), Transition(150), Transition(16217), Transition(16153), Transition(9114), Transition(3497), Transition(13759), Transition(5441), Transition(15941), Transversion(72), Transition(210), Deletion(8281), Transition(5082), Transition(13711), Transition(1119), Transition(13629), Transition(15346), Transition(14071), Transversion(7930)]
CIAO
H12a(1)
[Transition(8448), T

CIAO
L3a(4)
[Transition(769), Transition(16129), Transition(4104), Transition(3594), Transition(12816), Transition(7256), Transition(146), Transition(8468), Transition(16278), Transition(151), Transition(15301), Transition(10915), Transition(10664), Transition(13105), Transition(16311), Transversion(825), Transition(10810), Transition(16187), Transition(16316), Transition(11914), Transition(10688), Transition(13506), Transition(195), Transition(2885), Transition(2758), Transition(8655), Transition(13650), Transition(4312), Transition(13276), Transition(7521), Transition(16230), Transition(7146), Transition(16189), Transition(247), Transition(1018)]
CIAO
X2_225_153!(1)
[Transition(16256), Transition(14470), Transition(13966), Transition(143), Transition(16400), Transition(2836), Transition(6935), Transition(3744), Transition(11809), Transition(808), Transition(9518), Transition(3766), Transition(12346), Transition(3145), Transition(1719), Transition(6221), Transition(16207), Transition(

...

CIAO
M7a2(1)
[Transition(769), Transition(16129), Transition(4104), Transition(3594), Transition(2626), Transition(7256), Transition(146), Transition(15043), Transition(8468), Transition(16278), Transition(152), Transition(15301), Transition(10400), Transition(4386), Transition(10915), Transition(10664), Transition(13105), Transition(16311), Transversion(825), Transition(10810), Transition(16187), Transition(11914), Transition(15422), Transition(14783), Transition(10688), Transition(13506), Transition(195), Transition(2885), Transition(2758), Transition(6455), Transition(8655), Transition(16209), Transition(13650), Transition(2772), Transition(4312), Transition(13276), Transition(4958), Transition(9824), Transition(7521), Transition(12771), Transition(16230), Transition(489), Transition(7146), Transition(16189), Transition(247), Transition(1018)]
CIAO
M7a1(1)
[Transition(769), Transition(16129), Transition(4104), Transition(3594), Transition(2626), Transition(7256), Transition(146), Tr

L1b1a13(1)
[Transition(16129), Transition(5509), Transition(16264), Transition(11914), Transition(15115), Transition(16270), Transition(7055), Transition(5393), Transition(146), Transition(6548), Transition(5655), Transition(1438), Transition(10915), Transition(10664), Transition(6827), Transition(5036), Transition(13789), Transition(2352), Transition(14769), Transition(5046), Transversion(13880), Transversion(185), Transition(7867), Transition(709), Transition(710), Transition(1738), Transition(6989), Transition(2768), Transition(8248), Transition(3666), Transition(4312), Transition(13276), Transition(7389), Transition(182), Transition(14560), Transition(14178), Transition(357), Transition(16230), Transition(12519), Transition(3308), Transition(3693), Transition(14203), Transition(16126)]
CIAO
F1a1(1)
[Transition(8448), Transition(3970), Transition(15247), Transition(10771), Transition(16153), Transition(9114), Transition(16162), Transition(16172), Transition(16304), Transition(6962),

CIAO
M9a1a1c1a(1)
[Transition(769), Transition(16129), Transition(4104), Transition(11914), Transition(4491), Transition(3394), Transition(1041), Transition(146), Transition(15043), Transition(8468), Transition(16278), Transition(152), Transition(153), Transition(9242), Transition(15301), Transition(10400), Transition(16291), Transition(10664), Transition(4312), Transition(13105), Transition(7861), Transition(16311), Transversion(825), Transition(10810), Transition(16187), Transition(16316), Transition(3594), Transition(14783), Transition(10688), Transition(13506), Transition(195), Transition(2885), Transition(2758), Transition(8655), Transition(14417), Transition(13650), Transition(10915), Transition(7256), Transition(13276), Transition(7521), Transition(14308), Transition(7142), Transition(16230), Transition(489), Transition(7146), Transition(16189), Transition(16362), Transition(247), Transition(1018), Transition(16234)]
CIAO
U7a4a1(2)
[Transition(8448), Transition(15247), Transitio

CIAO
H45a(2)
[Transition(8448), Transition(8843), Transition(15247), Transition(10771), Transition(16153), Transition(9114), Transition(12192), Transition(16293), Transition(13759), Transversion(961), Transition(8898), Transversion(72), Transition(9039), Transition(210), Transition(5082), Transition(13711), Transition(14071), Transversion(7930), Transition(14587)]
CIAO
HV1b(1)
[Transition(16256), Transition(143), Transition(16400), Transition(12696), Transition(2836), Transition(14766), Transition(6935), Transition(3744), Transition(12705), Transition(808), Transition(9518), Transition(3766), Transition(12346), Transition(16067), Transition(11719), Transition(73), Transversion(8014), Transition(16207), Transition(9809), Transition(466), Transition(16083), Transition(11809), Transition(214), Transition(3421), Transition(16223), Transition(16355), Transition(7269), Transition(6374), Transition(103), Transition(8552), Transition(6632), Transition(15218), Transition(3145), Transition(6267)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/home/adm2/.local/lib/python2.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
